<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021_0316onematopea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prof. Kondo onomatopea
- date: 2021-0316
- file: 2021-0316onematopea.ipynb

近藤先生から頂いた課題の回答


In [ ]:
# word2vec データの読み込み
# ファイルの所在に応じて変更してください
w2v_base = './'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'

In [ ]:
#ひとつ下の '日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードする
from google.colab import files
uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする

# word2vec データの読み込み
# ローカルディスクから読み込むようになっています。colab でお使いの場合には適宜変更してください
# word2vec の訓練済モデルを入手
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz

In [ ]:
import os
import sys
import numpy as np
np.set_printoptions(precision=2)  # numpy の表示桁数設定

import json
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

# word2vec でーた処理のため gensim を使う
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from scipy import stats

import termcolor
import tqdm

# Colab では以下の 2 行の行頭の # を削除してから実行してください
!pip install jaconv
!pip install japanize_matplotlib
import jaconv  # ひらがなカタカナ変換用 `pip install jaconv` してください
import japanize_matplotlib  # matplotlib の日本語表示

# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータ
ccap_base = './'
#onomatopea_excel = '日本語オノマトペ辞典4500より.xlsx'
onomatopea_excel = '日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(ccap_base, onomatopea_excel), sheet_name='2761語')

In [ ]:
# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります
#w2v_base = '/Users/asakawa/study/2016wikipedia/'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 

w2v = asakawa_w2v

In [ ]:
onomatopea = list(set(sorted(onmtp2761['オノマトペ'])))

print('# オノマトペのうち，word2vec に登録があるかどうかを調査')
kana_entries, kata_entries = [], []
count = 0
for word in onomatopea:
    count += 1
    if word in w2v.vocab:
        kana_entries.append(word)

    kata_w = jaconv.hira2kata(word)
    if kata_w in w2v.vocab:
        kata_entries.append(kata_w)
        
entries = kana_entries + kata_entries

#print(len(kana_entries), len(kata_entries), len(onomatopea), count)
print('There are ', len(entries), ' in ', len(onomatopea), ' onomatopea words in word2vec from jawikipedia')
print('総数がオノマトペデータより多いのは，平仮名表記とカタカナ表記と両者で wikipedia に登録があった場合に重複してカウントしているからです。')
print('カタカナ オノマトペ総数:', len(kata_entries))
print('ひらがな オノマトペ総数:', len(kana_entries))


In [ ]:
test_n = 10  # 何語ランダムサンプリングするかを決める数
topn = 5    # 上位 何語を表示するかを決める数
print('確認用 {} 個のオノマトペをランダムサンプリングして表示。実行毎に結果が異なります:'.format(test_n))
for _ in range(test_n):
    word = np.random.choice(entries)
    print('単語:{0}, ID:{1}'.format(termcolor.colored(word,'green'), entries.index(word)), end=" ")
    print('word2vec の単純な最近隣語:',  termcolor.colored([x[0] for x in w2v.similar_by_word(word,topn=topn)],'blue'))

In [ ]:
print('ここからが本体')
class projection():
    """射影行列の作成
    オノマトペ空間への射影行列の作成
    
    引数:
    w2v: gensim.KeyedVectors()
        訓練済 word2vec データ
    wordlist: list
        射影空間を作成する語彙リスト
    """
    def __init__(self, w2v=w2v, wordlist=entries,topn=5):
        self.topn=10
        self.entries = wordlist
        self.w2v = w2v
        
        # 行列 P の初期化 entries 行，word2vec 次元 行の行列 
        self.X = np.zeros((len(set(self.entries)), w2v.vector_size), dtype=np.float)
        
        # 各行に word2vec ベクトルをコピー
        for i, x in enumerate(set(self.entries)):
            self.X[i] = np.copy(self.w2v[x])
        
        invX = np.linalg.inv(np.dot(self.X, self.X.T))   #  (X X^T)^{-1}
        self.P = np.dot(self.X.T, np.dot(invX, self.X))  # X^T (X X^T)^{-1} X 射影行列
        #self.P_ = np.dot(self.X.T, self.X)               # 規格化していない射影行列

        self.I = np.eye(len(self.P))              # 単位行列
        self.C = self.I - self.P                  # 直交補空間
        XXT = np.dot(self.X, self.X.T)       # shpae(dim, dim)
        iXXT = np.linalg.inv(XXT)            # shape(dim, dim)
        XTiXTX = np.dot(self.X.T, iXXT)      # shape(w2v_dim, dim)
        self.P = np.dot(XTiXTX,self.X)       # shape(w2v_dim, w2v_dim)
        self.CovP = np.dot(self.X.T,self.X)  # shape(w2v_dim, w2v_dim)
        self.CorP = np.corrcoef(self.X.T)    # shape(w2v_dim, w2v_dim)


    def prointProj(self, targets=['電車','ネコ','東京'], topn=5):
        """targets で指定された単語を射影したベクトルから得られる最近接語の印字"""
        for target in targets:
            if not target in self.w2v:
                continue
            x = np.array(self.w2v[target])
            Px = np.dot(self.P, x)
            print(target, ':--> ', end="")
            for word in self.w2v.similar_by_vector(Px,topn=topn):
                print(word[0], end=" ")
            print()
            Cx = np.dot(self.C, x)
            print('\t 直交補空間への射影:--> ', end='')
            for word in w2v.similar_by_vector(Cx,topn=topn):
                print(word[0], end=" ")
            print()
            
            
    def getProjWord(self, targets=['電車','ネコ','東京'], topn=5, Proj=None):
        """targets で指定された単語を射影したベクトルから得られる最近接語リストを返す"""
        if Proj == 'Prj':
            P = self.P
        if Proj == 'Cov':
            P = self.CovP
        elif Proj == 'Cor':
            P = self.CorP
        elif Proj == 'w2v':
            P = self.I
        else:
            P = self.P
        
        if topn == None:
            topn = self.topn

        ret = {}
        for target in targets:
            if not target in self.w2v:
                continue
            x = np.array(self.w2v[target])
            Px = np.dot(P, x)
            retdic = self.w2v.similar_by_vector(Px,topn=topn)
            ret[target] = [x[0] for x in retdic]
        return ret


    def __call__(self, targets=['魚', '肴', 'さかな', 'サカナ'], Proj=None,topn=5):
        if Proj == None:
            Proj = 'Prj'
        if topn == None:
            topn = self.topn
        return self.getProjWord(targets, Proj=Proj, topn=topn)


    
topn = 5
P = projection(topn=topn)

# 以下はテスト用
print(P.getProjWord())
word = 'ペチャクチャ'
print(termcolor.colored(word, 'green'), P.getProjWord([word, 'イヌ'],topn=topn))

#v = np.dot(P.CorP, w2v[word])
#print([x[0] for x in w2v.similar_by_vector(v)])

#v = np.dot(P.CovP, w2v[word])
#print([x[0] for x in w2v.similar_by_vector(v)])
for Proj in ['Prj', 'Cov', 'Cor', 'w2v']:
    print(Proj, P([word], Proj=Proj, topn=topn))


In [ ]:
print('# 近藤先生との相談で，各オノマトペの射影行列がどのくらいオノマトペを含むかを調べる')

topn = 1024  # 上位何語分を検討するかを決める数
#topn = 3
print('# 上位 {0} 最隣接語を対象とする。\nオノマトペ総数:{1}'.format(topn,len(entries)))

def hit_words(word_list):
    hit_list = []
    for w in word_list:
        if w in entries:
            hit_list.append(w)
    return hit_list


count_dic = {}
for i, word in tqdm.tqdm(enumerate(set(entries[:]))):
    #print('{:07d} words processed    \r'.format(i), end="")
    count_dic[word] = {}
    for Proj in ['Prj', 'Cov', 'Cor', 'w2v']:
        _list = P([word], Proj=Proj, topn=topn)
        hit_list = hit_words(_list[word])
        count_dic[word][Proj] = len(hit_list)

#print(json.dumps(count_dic, ensure_ascii=False, indent=2))

In [ ]:
#  結果を保存しローカルファイルにダウンロード
json_file = 'onematopea_count.json'
excel_file = 'onomatopea_count.xlsx'
with open(json_file, 'w') as f:
    json.dump(count_dic, f, ensure_ascii=False, indent=2)

df_json = pd.read_json(json_file).transpose()

df_json.to_excel(excel_file)

# ダウンロードする場合には，次行の行頭の # を削除してください。
#files.download(excel_file)
print(df_json.head())
df_json.tail()

In [ ]:
test_n = 5
all_words = list(w2v.vocab.keys())[1:]  # 最初は '</S>' なので除外
print('確認用 全 wikipedia.ja 登録単語数 {0} から {1} 語をランダムサンプリング'.format(len(all_words), test_n), end="")
print('実行毎に結果が異なります:\n')
for _ in range(test_n):
    word = np.random.choice(all_words)
    print('ターゲット単語:{0}, 射影ベクトルの最近隣語:{1}'.format(
        termcolor.colored(word,'green'), 
        termcolor.colored(P([word], topn=3),'blue')))
